# RAFT방식의 Finetuning

In [ ]:
import torch
import torchvision

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA available?:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

torch: 2.8.0.dev20250319+cu128
torchvision: 0.22.0.dev20250319+cu128
CUDA available?: True
GPU: NVIDIA A100-SXM4-80GB


## 환경설정

transformers, datasets, accelerate, trl, peft 라이브러리를 설치합니다.

* **transformers**: 대형 언어모델(LLM) 활용과 파인튜닝을 위한 핵심 라이브러리입니다.
* **datasets**: 다양한 데이터셋을 쉽게 불러오고 전처리하는 데 사용합니다.
* **accelerate**: 멀티GPU 및 분산 학습, 메모리 최적화 기능을 지원합니다.
* **trl**: SFT(지도학습), RLHF 등 LLM 미세조정용 Trainer와 도구를 제공합니다.
* **peft**: LoRA 등 저비용 파인튜닝(PEFT) 기법을 적용할 때 사용합니다.

In [ ]:
%pip install -U transformers datasets accelerate trl peft tiktoken protobuf sentencepiece scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import transformers, datasets, accelerate, trl, peft

print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("accelerate:", accelerate.__version__)
print("trl:", trl.__version__)
print("peft:", peft.__version__)

transformers: 4.52.4
datasets: 3.6.0
accelerate: 1.8.0
trl: 0.18.2
peft: 0.15.2


## 문서준비 및 병합
- klue_mrc_prompt_docs_5_answer_citations.csv
- klue_mrc_prompt_docs_1_4_answer_citations.csv
- klue_mrc_nominal_question_docs_1_5_answer_citations.csv
- klue_mrc_prompt_multidocs_answer_citations.csv

In [1]:
# klue_mrc_prompt_docs_5_answer_citations.csv
!gdown 1cybA7CWnNg9e_M73kRkVIi8nA9TG3co7
# klue_mrc_prompt_docs_1_4_answer_citations.csv
!gdown 1rikKQUmaN5yvFGcQi5xhfUFLR3slElmK
# klue_mrc_nominal_question_docs_1_5_answer_citations.csv
!gdown 1Ifvk2IFHB0x07G92P2HZHbpZhKIBAaML
# klue_mrc_prompt_multidocs_answer_citations.csv
!gdown 1TAQw4g4py9hrCAKNGG6DuTxDqgUiPCO_

Downloading...
From: https://drive.google.com/uc?id=1cybA7CWnNg9e_M73kRkVIi8nA9TG3co7
To: /content/klue_mrc_prompt_docs_5_answer_citations.csv
100% 7.45M/7.45M [00:00<00:00, 56.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rikKQUmaN5yvFGcQi5xhfUFLR3slElmK
To: /content/klue_mrc_prompt_docs_1_4_answer_citations.csv
100% 4.10M/4.10M [00:00<00:00, 174MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ifvk2IFHB0x07G92P2HZHbpZhKIBAaML
To: /content/klue_mrc_nominal_question_docs_1_5_answer_citations.csv
100% 3.37M/3.37M [00:00<00:00, 251MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TAQw4g4py9hrCAKNGG6DuTxDqgUiPCO_
To: /content/klue_mrc_prompt_multidocs_answer_citations.csv
100% 5.33M/5.33M [00:00<00:00, 27.0MB/s]


In [2]:
import pandas as pd
import ast

# 파일명과 타입 정의
file_info = [
    ('klue_mrc_prompt_docs_5_answer_citations.csv', 1),
    ('klue_mrc_prompt_docs_1_4_answer_citations.csv', 2),
    ('klue_mrc_nominal_question_docs_1_5_answer_citations.csv', 3),
    ('klue_mrc_prompt_multidocs_answer_citations.csv', 4),
]

# 통일할 컬럼명
final_columns = ['question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']

# 파일별로 읽어서 정리
dfs = []
for fname, type_id in file_info:
    df = pd.read_csv(fname)
    df['type'] = type_id
    # 컬럼명 통일 (nominal_question, question_only → question으로 맞추기)
    for candidate in ['nominal_question', 'question_only']:
        if candidate in df.columns and 'question' not in df.columns:
            df = df.rename(columns={candidate: 'question'})
    # 필요한 컬럼만 선택 (일부 파일에서 누락될 수 있음)
    cols = [col for col in final_columns if col in df.columns] + ['type']
    df = df[cols]
    # 누락된 컬럼은 빈 값으로 채우기
    for col in final_columns:
        if col not in df.columns:
            df[col] = ''
    # 최종 컬럼 순서 적용
    df = df[final_columns + ['type']]
    dfs.append(df)

# 하나로 합치기
df_all = pd.concat(dfs, ignore_index=True)

# docs와 doc_citations 컬럼을 리스트로 변환
for col in ['docs', 'doc_citations']:
    df_all[col] = df_all[col].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x and x != '' else []
    )

# 결과 확인
print(df_all.shape)
print(type(df_all.loc[0, 'docs']), df_all.loc[0, 'docs'])            # <class 'list'>
print(type(df_all.loc[0, 'doc_citations']), df_all.loc[0, 'doc_citations'])  # <class 'list'>
df_all.head()


(972, 7)
<class 'list'> ['1994년 11월, 배타적 경제수역(EEZ)을 정한 유엔 해양법 협약의 발효로 바다 관할권이 12해리에서 200해리까지 확대되었으나 한·일간의 거리가 400해리가 채 되지 않아 바다 경계선을 별도로 정해야 했다. 일본은 1994년 발효된 유엔해양법협약에 근거해 근해의 작은 섬들을 직선으로 연결, 영해기선을 새롭게 설정하고, 1997년 1월 1일을 기해 시행에 들어갔다. 1997년 5월 당시 유종하 외무장관은 미국방문 길에 수행기자단과의 간담회에서 “어업문제에 대한 일본의 요구가 더 이상은 버티기 어려운 상황에 이르러 있다”고 토로하기도 했다.\n1996년 5월에 김영삼정부는 울릉도와 일본 오키 섬의 중간부분을 EEZ의 경계로 발표해 독도를 우리측 수역에 포함시킨다는 성명을 발표했었다. 1997년에 들어서 일본은 "자신들의 직선기선을 넘어왔다"며 한국의 어선들을 무차별적으로 나포하였고 일본의 하시모토 류타로 총리까지 나서서 "한국어선 나포는 정당하다"며 어선나포를 비호했고 일본은 한국정부에 "직선기선을 인정하지 않으면 어업협정을 파기하겠다"며 협박했다. 97년 7월 한국이 배타적 경제수역(EEZ)의 기점을 울릉도로 하는 발표를 한다. 7월 29일, 한국은 일본의 직선기선 인정 요구에 사실상 굴복하게 되었다. 그 이후에도 독도에 대한 수역을 제외하고 한국과 일본의 어업협상이 계속되었고 서로간의 마찰은 더더욱 심화되었다. 일각에서는 1997년 10월 당시 김영삼 정권은 잠정공동수역안(잠정조치수역은 독도 중간수역)을 공식적으로 받아들여 독도를 중간수역으로 하기로 일본과 합의하였다고 하지만, 당시 뉴스와 자료들 검색 결과 1997년 10월 10일, 도쿄에서 제6차 한일 실무회담을 진행하고 있었고 당시 한일양국은 당시 독도 주변수역을 제외하고 협상에 임하였고 당시 정부당국이 "독도 영유권이 훼손되지 않는 조건하에 이 안을 철회한다"며 독도에 대한 영유권주장은 변함이 없었다.(참조) \n1997년 10월 22일, 일본측은 독도

,question,docs,user_prompt,system_prompt,answer,doc_citations,type
0,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,"[1994년 11월, 배타적 경제수역(EEZ)을 정한 유엔 해양법 협약의 발효로 바...",질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?\n\ndoc...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",북태평양 기단과 오호츠크해 기단이 만나 형성되는 장마전선은 한반도 중남부를 오르내리...,[doc2],1
1,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,[아이티센(124500)그룹의 쌍용정보통신(010280)이 해군전술C4I체계 성능개...,질문: 성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?\n\ndocs...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는 과학기술정보통신부입니다...,[doc5],1
2,개막전에서 3안타 2실점을 기록해서 패한 선수는?,[시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인...,질문: 개막전에서 3안타 2실점을 기록해서 패한 선수는?\n\ndocs:\ndoc1...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",개막전에서 3안타 2실점을 기록하여 패한 선수는 사이타마 세이부 라이온스의 와쿠이 ...,[doc1],1
3,컵라면 매출에서 불닭볶음면을 이긴 상품은?,[영화관 안에서만 주로 접했던 팝콘이 편의점 진열대의 윗자리를 차지하는 대표 스낵으...,질문: 컵라면 매출에서 불닭볶음면을 이긴 상품은?\n\ndocs:\ndoc1: 영화...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",컵라면 매출에서 불닭볶음면을 이긴 상품은 GS25의 '오모리 김치찌개라면'과 세븐일...,[doc5],1
4,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,[포브스는 최근 “석탄은 기원전 315년 그리스 문헌에 대장간에서 원료로 쓰던 기록...,질문: 정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?\n\ndocs:\...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",정부에게 환경과 관련하여 우선적으로 원조를 받고 있는 곳은 중국입니다. 중국 정부는...,[doc2],1


In [ ]:
print("type별 데이터 분포:")
for t, n in df_all['type'].value_counts().sort_index().items():
    print(f"type {t}: {n}건")

type별 데이터 분포:
type 1: 469건
type 2: 285건
type 3: 193건
type 4: 462건


In [ ]:
from sklearn.model_selection import train_test_split

train_dfs = []
test_dfs = []

for t in sorted(df_all['type'].unique()):
    sub = df_all[df_all['type'] == t].sample(frac=1, random_state=42).reset_index(drop=True)  # 셔플
    train, test = train_test_split(sub, test_size=0.2, random_state=42)
    train_dfs.append(train)
    test_dfs.append(test)

# 합치기
df_train = pd.concat(train_dfs, ignore_index=True)
df_test = pd.concat(test_dfs, ignore_index=True)

print(f"train 데이터 shape: {df_train.shape}")
print(f"test 데이터 shape: {df_test.shape}")
print(df_train['type'].value_counts().sort_index())
print(df_test['type'].value_counts().sort_index())

train 데이터 shape: (1126, 7)
test 데이터 shape: (283, 7)
type
1    375
2    228
3    154
4    369
Name: count, dtype: int64
type
1    94
2    57
3    39
4    93
Name: count, dtype: int64


## OpenAI 형식 변환 및 jsonl 저장

In [ ]:
system_prompt = """당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요."""

# OpenAI format으로 데이터 변환을 위한 함수
def format_data(row):
    """
    DataFrame의 row에서 OpenAI 파인튜닝용 messages 구조(dict)를 생성합니다.
    """
    docs = row['docs']
    # 문서에 번호 붙이기
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(docs)]
    docs_str = '\n'.join(doc_items)

    user_prompt = f"""질문: {row['question']}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""

    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": str(row['answer']).strip()}
        ]
    }


In [ ]:
import json

# 변환
train_data = df_train.apply(format_data, axis=1).tolist()
test_data = df_test.apply(format_data, axis=1).tolist()

# 저장
with open('klue_mrc_rag_train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

with open('klue_mrc_rag_test.jsonl', 'w', encoding='utf-8') as f:
    for item in test_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')


In [ ]:
print(json.dumps(train_data[0], ensure_ascii=False, indent=2))

{
  "messages": [
    {
      "role": "system",
      "content": "당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.\n아래 지침을 반드시 지켜주세요:\n\n- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.\n- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.\n- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.\n- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.\n- 모든 답변은 존댓말을 사용하세요."
    },
    {
      "role": "user",
      "content": "질문: 멕시코 연방 정부는 메탈클래드에게 얼마를 보상하라고 하였나?\n\ndocs:\ndoc1: 소송가액만 약 5조원에 달하는 론스타펀드가 한국 정부를 상대로 제기한 투자자-국가소송(ISD)의 첫 심리가 15일(현지시간) 미국 워싱턴 국제투자분쟁해결센터(ICSID)에서 열린다. ISD는 1987년 애플 홍콩법인이 스리랑카 정부를 상대로 낸 것을 시작으로 주로 선진국 투자자들이 개발도상국 정부를 제소해왔다.한국 정부가 ISD에 휘말린 것은 이번이 처음이다. 결과가 미칠 파장이 만만치 않은 만큼 정부는 국무조정실 주도로 태스크포스를 구성했다. 세금 회피를 위해 벨기에 등에 페이퍼컴퍼니를 세운 뒤 외환은행을 인수한 론스타는 소송 주체도 페이퍼컴퍼니를 내세운 것으로 알려져 정부는 이 점을 집중 공략할 것으로 전해졌다.○한국 정부의 첫 ISD 사건론스타가 ICSID에 중재를 신청한 것은 2012년 11월21일이다. 신청인(원고)은 LSF-KEB홀딩스, 스타홀딩

In [ ]:
# 한 줄에 하나의 json (OpenAI 형식)
train_data = []
with open('klue_mrc_rag_train.jsonl', encoding='utf-8') as f:
    for line in f:
        train_data.append(json.loads(line))

# test도 동일
test_data = []
with open('klue_mrc_rag_test.jsonl', encoding='utf-8') as f:
    for line in f:
        test_data.append(json.loads(line))

In [ ]:
train_data[0]['messages']

[{'role': 'system',
  'content': '당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.\n아래 지침을 반드시 지켜주세요:\n\n- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.\n- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.\n- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.\n- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.\n- 모든 답변은 존댓말을 사용하세요.'},
 {'role': 'user',
  'content': '질문: 멕시코 연방 정부는 메탈클래드에게 얼마를 보상하라고 하였나?\n\ndocs:\ndoc1: 소송가액만 약 5조원에 달하는 론스타펀드가 한국 정부를 상대로 제기한 투자자-국가소송(ISD)의 첫 심리가 15일(현지시간) 미국 워싱턴 국제투자분쟁해결센터(ICSID)에서 열린다. ISD는 1987년 애플 홍콩법인이 스리랑카 정부를 상대로 낸 것을 시작으로 주로 선진국 투자자들이 개발도상국 정부를 제소해왔다.한국 정부가 ISD에 휘말린 것은 이번이 처음이다. 결과가 미칠 파장이 만만치 않은 만큼 정부는 국무조정실 주도로 태스크포스를 구성했다. 세금 회피를 위해 벨기에 등에 페이퍼컴퍼니를 세운 뒤 외환은행을 인수한 론스타는 소송 주체도 페이퍼컴퍼니를 내세운 것으로 알려져 정부는 이 점을 집중 공략할 것으로 전해졌다.○한국 정부의 첫 ISD 사건론스타가 ICSID에 중재를 신청한 것은 2012년 11월21일이다. 신청인(원고)은 LSF-KEB홀딩스, 스타홀딩스 등 8곳이다. 이들 법인의 근거지는 룩셈부르크 한 곳, 나머지는 벨기에다. 뚜렷한 실체가

In [ ]:
from datasets import Dataset   # Hugging Face Datasets 라이브러리에서 Dataset 클래스를 가져옴.

# 리스트 형태로 준비된 train_dataset, test_dataset을 Hugging Face의 Dataset 객체로 변환.
# Dataset 객체는 데이터 로딩·샘플링·배치 작업 등 다양한 기능을 효율적으로 지원
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

print(type(train_dataset))
print(type(test_dataset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
train_dataset[0]   # 전처리·포맷팅된 학습 데이터가 정확히 기대하는 구조와 내용(messages, role, content 등)으로 정상적으로 만들어졌는지 확인

{'messages': [{'content': '당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.\n아래 지침을 반드시 지켜주세요:\n\n- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.\n- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.\n- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.\n- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.\n- 모든 답변은 존댓말을 사용하세요.',
   'role': 'system'},
  {'content': '질문: 멕시코 연방 정부는 메탈클래드에게 얼마를 보상하라고 하였나?\n\ndocs:\ndoc1: 소송가액만 약 5조원에 달하는 론스타펀드가 한국 정부를 상대로 제기한 투자자-국가소송(ISD)의 첫 심리가 15일(현지시간) 미국 워싱턴 국제투자분쟁해결센터(ICSID)에서 열린다. ISD는 1987년 애플 홍콩법인이 스리랑카 정부를 상대로 낸 것을 시작으로 주로 선진국 투자자들이 개발도상국 정부를 제소해왔다.한국 정부가 ISD에 휘말린 것은 이번이 처음이다. 결과가 미칠 파장이 만만치 않은 만큼 정부는 국무조정실 주도로 태스크포스를 구성했다. 세금 회피를 위해 벨기에 등에 페이퍼컴퍼니를 세운 뒤 외환은행을 인수한 론스타는 소송 주체도 페이퍼컴퍼니를 내세운 것으로 알려져 정부는 이 점을 집중 공략할 것으로 전해졌다.○한국 정부의 첫 ISD 사건론스타가 ICSID에 중재를 신청한 것은 2012년 11월21일이다. 신청인(원고)은 LSF-KEB홀딩스, 스타홀딩스 등 8곳이다. 이들 법인의 근거지는 룩셈부르크 한 곳, 나머지는 벨기에다. 뚜렷한 실체가 없는

## BaseModel - NCSOFT/Llama-VARCO-8B-Instruct

* **Hugging Face Model Hub**에 등록된
  **NCSOFT**(엔씨소프트)에서 공개한
  **Llama 계열 8B(약 80억 파라미터)의 Instruction(지시문 튜닝) 기반 언어모델**입니다.


**각 단어의 의미**

* **NCSOFT/**

  * 모델을 공개한 개발사 또는 연구기관(Hugging Face에서 계정 또는 조직 이름)
* **Llama-VARCO-8B**

  * Meta Llama 기반 구조 + VARCO라는 엔씨 고유 튜닝 전략이 적용된 8B(80억 파라미터) 모델임을 뜻합니다.
* **Instruct**

  * 단순 언어모델(base)이 아니라,
    "명령어/지시문(prompt)에 답변을 잘 하도록 추가로 파인튜닝된 모델"이라는 뜻입니다.
  * 즉, ChatGPT나 챗봇처럼 "질문하면 답하는" 데 최적화된 버전입니다.



### torch\_dtype
**torch\_dtype**는 모델을 메모리에 올릴 때 사용할 \*\*파라미터 데이터 타입(숫자 표현 방식)\*\*을 지정하는 옵션입니다.

**주요 옵션 값과 의미**

* **torch.float32**

  * 32비트 부동소수점(가장 정밀, 가장 많은 메모리 사용)
  * 전통적으로 가장 많이 사용

* **torch.float16**

  * 16비트 부동소수점(메모리 절약, 빠른 연산, 일부 정밀도 손실 가능)
  * 최신 GPU에서 대형 모델 학습/추론 시 널리 사용

* **torch.bfloat16**

  * 16비트 부동소수점이지만 float16보다 더 넓은 표현 범위 제공(정밀도 손실 적음)
  * NVIDIA A100, H100 등 최신 GPU에서 매우 효율적
  * 속도·메모리 효율은 float16처럼 좋고, 안정성은 float32에 가깝다

**실전에서 왜 쓸까?**

* 모델이 너무 커서 GPU 메모리가 부족할 때
  torch\_dtype를 float16, bfloat16으로 설정하면
  **메모리 사용량을 절반 이하로 줄일 수 있음**
* 연산 속도도 크게 빨라집니다.
* 최신 모델/실습에서는 float32보다
  bfloat16 또는 float16이 기본으로 쓰입니다.

**정리:**
torch\_dtype는
**모델의 크기, 연산 속도, 실습 환경(GPU) 최적화를 위해
모델 파라미터가 어떤 숫자 타입으로 올라갈지 정하는 중요한 옵션**입니다.
(메모리 부족/속도 저하 시, float16/bfloat16으로 바꾸면 실습이 쉬워집니다.)


In [ ]:
# Hugging Face transformers 라이브러리의 AutoModelForCausalLM 클래스. "Causal Language Model"(GPT류, Llama류 등) 텍스트 생성용 사전학습 모델을 이름만 지정하면 자동으로 불러오는 도구.
# AutoTokenizer 클래스는 동일 사전학습 모델에 맞는 토크나이저를 자동으로 불러옵니다.텍스트를 숫자 토큰으로 바꾸고, 다시 텍스트로 되돌릴 때 사용합니다.
from transformers import AutoModelForCausalLM, AutoTokenizer

# 사용할 사전학습(프리트레인드) 언어모델의 이름.
# Hugging Face Hub에 등록된 Llama-VARCO-8B-Instruct 모델을 불러옵니다.
pretrained_model_name = "NCSOFT/Llama-VARCO-8B-Instruct"

# Hugging Face의 AutoModelForCausalLM 클래스를 사용해, pretrained_model_name에 해당하는 사전학습 모델을 로딩.
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name,
    device_map="auto",           # GPU 환경을 자동 감지해 최적의 디바이스에 모델을 할당
    torch_dtype=torch.bfloat16,  # 모델 파라미터를 bfloat16(16비트 부동소수점) 형식으로 올려, 메모리 효율과 연산 속도를 높임.
)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)  # 같은 pretrained_model_name을 사용해 해당 모델의 토크나이저도 함께 불러옴.
                                                                  # 토크나이저는 텍스트를 숫자 토큰으로 변환하고, 생성 결과를 다시 텍스트로 복원하는 역할을 수행.

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Llama-3, Llama-VARCO 등 최신 챗봇 계열 모델에서 여러 메시지(system/user/assistant 등)를 채팅 프롬프트 형식의 한 줄 텍스트로 자동 변환
# 첫 번째 학습 데이터의 messages 리스트(여러 역할별 대화)를 입력값으로 사용
# tokenize=False: 변환 결과를 숫자 토큰이 아니라 **사람이 읽을 수 있는 텍스트(문자열)**로 반환.
# add_generation_prompt=False: 모델에게 "여기서부터 답변을 생성하라"는 추가 프롬프트를 붙이지 않습니다.
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

질문: 멕시코 연방 정부는 메탈클래드에게 얼마를 보상하라고 하였나?

docs:
doc1: 소송가액만 약 5조원에 달하는 론스타펀드가 한국 정부를 상대로 제기한 투자자-국가소송(ISD)의 첫 심리가 15일(현지시간) 미국 워싱턴 국제투자분쟁해결센터(ICSID)에서 열린다. ISD는 1987년 애플 홍콩법인이 스리랑카 정부를 상대로 낸 것을 시작으로 주로 선진국 투자자들이 개발도상국 정부를 제소해왔다.한국 정부가 ISD에 휘말린 것은 이번이 처음이다. 결과가 미칠 파장이 만만치 않은 만큼 정부는 국무조정실 주도로 태스크포스를 구성했다. 세금 회피를 위해 벨기에 등에 페이퍼컴퍼니를 세운 뒤 외환은행을 인수한 론스타는 소송 주체도 페이퍼컴퍼니를 내세운 것으로 알려져 정부는 이 점을 집중 공략할 것으로 전해졌다.○한국 정부의 첫 ISD 사건론스타가 ICSID에 중재를 신청한 것은 2012년 11월21일이다. 신청인(원고)은 LSF-KEB홀딩스, 스타홀딩스 등 8곳이다. 이들 법인의 근

## LoRA Finetuning

**LoRA란?**

* LoRA는 \*\*"Low-Rank Adapter(저랭크 어댑터)"\*\*의 줄임말입니다.
* 거대한 대형언어모델(LLM)의 **전체 파라미터를 일일이 미세조정(파인튜닝)하지 않고**,
  **딱 필요한 핵심 부분만 저렴하게 빠르게 학습**하는 최신 파인튜닝 방법입니다.

**왜 LoRA가 등장했을까?**

* GPT, Llama, DeepSeek 같은 대형언어모델은 **파라미터(매개변수) 수가 수십억\~수조 개**나 됩니다.
* 이런 모델을 파인튜닝하려면 **막대한 GPU 메모리와 시간, 저장 공간**이 필요합니다.
* 하지만, 실제로 특정 태스크에 맞게 모델을 조정할 때 **전체를 다 바꿀 필요가 없습니다.**
* 대부분의 정보는 기존 모델에 이미 들어있고,
  **특정 입력(질문)과 특정 출력(답변)의 관계만 살짝 조정**해주면 충분합니다.

**LoRA의 원리**

* 기존 대형 모델의 핵심 연산(주로 "곱셈" 부분)에
  \*\*작고 얇은 "보조 네트워크(어댑터 레이어)"\*\*를 덧붙입니다.
* 전체 모델은 거의 건드리지 않고,
  **이 어댑터 레이어의 파라미터만 새로 추가해서 학습**합니다.
* 학습이 끝나면,

  * 원본 모델은 그대로
  * 어댑터(작은 추가 파라미터)만 별도로 저장하면 끝!
* 추론할 땐 **원본 모델 + LoRA 어댑터**를 합쳐서 쓸 수 있습니다.

**LoRA의 장점**

* **파인튜닝 비용(시간, 메모리, 저장 용량)이 압도적으로 절약**됩니다.
* 7B, 13B, 70B 등 대형 모델도
  **일반 GPU(24GB/48GB)로도 쉽게 파인튜닝**이 가능합니다.
* **동일한 원본 모델에 다양한 LoRA 어댑터만 바꿔 끼우며
  다양한 분야별 파인튜닝 결과를 쉽게 쓸 수 있습니다.**

**LoRA와 기존 방식의 비교**

* **기존 파인튜닝:**
  전체 파라미터(수십\~수백 GB)를 새로 저장/관리/학습 → 비효율적
* **LoRA:**
  원본은 그대로 두고,
  변화가 필요한 부분(수 MB\~수십 MB)만 별도로 학습/저장



**실전에서의 활용 예시**

* 번역 LoRA, 요약 LoRA, 감정분석 LoRA 등
  **하나의 원본 모델에 여러 용도별 어댑터를 저장/관리**할 수 있습니다.
* **A100 80GB, 3090, T4 등 다양한 GPU 환경에서도
  고성능 LLM 튜닝이 매우 쉽게 가능합니다.**

**정리:**
LoRA는
“LLM의 힘은 그대로,
비용/시간/메모리/유지보수는 최소로”
파인튜닝을 할 수 있게 해주는
AI 실무에서 가장 중요한 기법 중 하나입니다.


In [ ]:
# PEFT 라이브러리에서 LoRA 어댑터의 구조와 동작 방식을 설정하는 클래스를 가져옵
from peft import LoraConfig

# LoRA 어댑터를 어떻게 적용할지 상세하게 지정하는 설정 객체
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

### LoRAConfig 주요 매개변수

| 매개변수                | 의미/역할                                         | 주요 옵션·예시                                                                                                             |
| ------------------- | --------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **lora\_alpha**     | LoRA 어댑터의 학습 스케일 팩터(변화 강도). 값이 크면 학습 변화가 완만해짐 | 16, 32, 64 등                                                                                                         |
| **lora\_dropout**   | 어댑터에만 적용되는 드롭아웃 확률. 과적합 방지                    | 0.05, 0.1 등                                                                                                          |
| **r**               | LoRA 어댑터의 랭크(정보량/두께). 값이 크면 더 많은 정보, 메모리 사용↑  | 8, 16, 32 등                                                                                                          |
| **bias**            | 기존 모델의 bias 파라미터도 LoRA로 튜닝할지 여부               | "none"(권장), "all"                                                                                                    |
| **target\_modules** | LoRA를 어떤 레이어(부분)에 적용할지 지정                     | "q\_proj", "v\_proj", "o\_proj", "up\_proj" 등<br>(모델 구조에 따라 다름)                                                      |
| **task\_type**      | LoRA가 적용될 문제 유형(파인튜닝 목적)                      | "CAUSAL\_LM"(생성), "SEQ\_CLS"(분류),<br>"SEQ\_2\_SEQ\_LM"(번역/요약),<br>"TOKEN\_CLS"(토큰분류),<br>"QUESTION\_ANSWERING"(질의응답) |


#### target\_modules 주요 옵션 예시

* `"q_proj"`: Attention의 Query projection, 효율/성능 균형, 기본 선택
* `"v_proj"`: Attention의 Value projection, 기본 선택
* `"k_proj"`, `"o_proj"`: 필요 시 성능 추가용
* `"up_proj"`, `"down_proj"`, `"gate_proj"`: FFN 계열, 모델 구조/실험 목적 따라 추가

#### task\_type 주요 옵션 예시

* `"CAUSAL_LM"`: 텍스트 생성(챗봇, GPT류)
* `"SEQ_CLS"`: 문장/문단 분류(감정, 카테고리)
* `"SEQ_2_SEQ_LM"`: 입력→출력 변환(번역, 요약 등)
* `"TOKEN_CLS"`: 단어 단위 분류(개체명 인식 등)
* `"QUESTION_ANSWERING"`: 질문-정답 위치 예측(문서 내 Q\&A)


**정리:**
이 매개변수들은
**LoRA 어댑터의 구조, 적용 범위, 정보량, 목적을 한 번에 결정하는
핵심 설정값**입니다.
(실제 모델/태스크/실험 목표에 맞게 조합해서 사용하세요.)


In [ ]:
# TRL(Transformers Reinforcement Learning) 라이브러리에서 SFT(지도 미세조정) 실험의 각종 설정을 담을 수 있는 클래스
from trl import SFTConfig

max_seq_length=8192  # 한 입력(프롬프트+정답)에 허용되는 최대 토큰 길이

# 아래 옵션들을 한 번에 묶어 학습 전체 설정을 관리
args = SFTConfig(
    output_dir="llama3-8b-rag-ko",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=[],
    max_seq_length=max_seq_length,
    label_names=["labels"],
)

### SFTConfig 주요 매개변수

| 매개변수                                | 설명                                                  | 주요 예시 값/옵션                              |
| ----------------------------------- | --------------------------------------------------- | --------------------------------------- |
| **output\_dir**                     | 결과 모델/로그를 저장할 경로 또는 저장소 ID                          | `"./results"`                           |
| **num\_train\_epochs**              | 전체 데이터를 몇 번 반복 학습할지(에포크 수)                          | `3`, `5`                                |
| **per\_device\_train\_batch\_size** | 각 GPU(디바이스)에서 한 번에 입력할 데이터 수(배치 크기)                 | `2`, `4`, `8`                           |
| **gradient\_accumulation\_steps**   | 여러 미니배치를 모아 한 번에 업데이트(실질 배치 크기 키우기)                 | `1`, `2`, `4`                           |
| **gradient\_checkpointing**         | 메모리 절약 기능(필요할 때만 중간 계산값 저장)                         | `True`, `False`                         |
| **optim**                           | 최적화 알고리즘(학습 방법)                                     | `"adamw_torch_fused"`                   |
| **logging\_steps**                  | 몇 step마다 로그를 출력할지                                   | `10`, `50`, `100`                       |
| **save\_strategy**                  | 모델 저장 방식(주기)                                        | `"steps"`, `"epoch"`                    |
| **save\_steps**                     | 몇 step마다 모델을 저장할지                                   | `50`, `100`                             |
| **bf16**                            | bfloat16 연산 사용(GPU 메모리 절약)                          | `True`, `False`                         |
| **learning\_rate**                  | 파라미터 업데이트 속도(학습률)                                   | `1e-4`, `5e-5`                          |
| **max\_grad\_norm**                 | 그래디언트 클리핑 임계값(학습 안정화)                               | `0.3`, `1.0`                            |
| **warmup\_ratio**                   | 워밍업 단계 비율(초기 학습률 천천히 증가)                            | `0.03`, `0.1`                           |
| **lr\_scheduler\_type**             | 학습률 조정 방식                                           | `"constant"`, `"linear"`                |
| **push\_to\_hub**                   | 학습 결과를 Hugging Face Hub로 업로드할지 여부                   | `True`, `False`                         |
| **hub\_model\_id**                  | 업로드할 Hugging Face Hub 저장소 ID                        | `"nugunaai/llama3-8b-news-analyzer-ko"` |
| **hub\_token**                      | Hugging Face Hub 인증 토큰 사용 여부                        | `True`                                  |
| **remove\_unused\_columns**         | 학습에 안 쓰는 데이터 컬럼 자동 제거 여부                            | `True`, `False`                         |
| **dataset\_kwargs**                 | 데이터셋 추가 옵션(딕셔너리 형태)                                 | `{"skip_prepare_dataset": True}`        |
| **report\_to**                      | 학습 로그를 기록할 대상(예: wandb, tensorboard, 빈 리스트면 기록 안 함) | `[]`, `["wandb"]`                       |
| **max\_seq\_length**                | 한 입력에 허용되는 최대 토큰(단어) 수                              | `2048`, `4096`, `8192`                  |
| **label\_names**                    | Trainer가 label로 인식할 컬럼명                             | `["labels"]`                            |


### data\_collator 함수

* 미니배치(batch) 데이터를 모델이 바로 학습할 수 있는 형태(토큰·마스크·정답)로 변환합니다.
* 특히 아래와 같은 LLaMA-3 채팅 포맷을 쓸 때,
  “어디까지가 질문/어디서부터가 답변(assistant)인지”를 정확히 구분해서
  모델이 정답(답변 부분)만 학습하도록 레이블을 지정합니다.

#### LLaMA-3 채팅 포맷

* LLaMA-3 채팅 포맷은
  **LLaMA-3 계열 챗봇 모델이 대화 내용을 이해하고 답변할 수 있도록 만들어진 입력 데이터 구조**입니다.
* 여러 역할(시스템, 유저, 어시스턴트)의 메시지를
  **특별한 토큰과 구조**로 묶어서 하나의 프롬프트로 합치는 방식입니다.


##### 구조 예시

아래와 같이 대화 흐름을 명확히 구분하는 토큰들이 사용됩니다:

```
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
[시스템 역할 지침]<|eot_id|>
<|start_header_id|>user<|end_header_id|>
[유저 질문]<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
[모델의 답변]<|eot_id|>
```

* `<|begin_of_text|>` : 전체 프롬프트의 시작을 알리는 토큰
* `<|start_header_id|>role<|end_header_id|>` : 각 메시지의 역할 구분(시스템, 유저, 어시스턴트 등)
* 각 메시지 끝에 `<|eot_id|>` : 하나의 메시지 블록이 끝났음을 알림

#### 왜 이 포맷이 필요할까?

* 모델이 **“어디까지가 시스템 안내, 어디서부터가 유저 질문, 어디서부터가 답변인지”**
  정확하게 파악할 수 있습니다.
* 여러 턴(turn)의 대화가 이어질 때도
  메시지 경계를 명확히 구분해 혼동 없이 맥락을 유지할 수 있습니다.
* LLaMA-3 계열 모델은 이런 포맷으로 학습되어 있기 때문에
  **실전 파인튜닝/추론 시에도 반드시 이 구조로 입력해야**
  기대하는 챗봇 성능을 발휘할 수 있습니다.

#### 실제 실습/프로그래밍에서

* 데이터 전처리에서 여러 메시지를
  위와 같은 포맷으로 하나의 텍스트로 합친 후,
  토크나이즈해서 모델에 입력합니다.
* assistant(정답) 부분만 레이블로 지정하면
  모델이 질문-답변 형식 대화에 최적화됩니다.

**정리:**
LLaMA-3 채팅 포맷은
“모델이 다양한 역할의 대화 맥락을 혼동 없이 이해하고
최적의 답변을 내놓을 수 있도록
특별한 토큰 구조로 입력을 포맷팅하는 방식”입니다.

In [ ]:
def data_collator(batch):

    # 미니배치(batch) 데이터를 저장할 딕셔너리
    # "input_ids": 각 샘플의 숫자 토큰 시퀀스(모델 입력)
    # "attention_mask":	input_ids에서 실제 데이터는 1, 패딩은 0으로 구분(모델이 무시해야 할 부분 표시)
    # "labels":	학습 정답(정답이 아닌 위치는 -100, assistant(답변) 구간만 정답 토큰 값으로 채움)
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    # 미니배치(batch) 안에 있는 각 예시 데이터를 하나씩 처리
    for example in batch:
        messages = example["messages"]                   # 예시 데이터에서 메시지 리스트(system, user, assistant 등)를 꺼냄.

        # LLaMA 3 채팅 템플릿 적용 (시작 토큰 포함)
        prompt = "<|begin_of_text|>"                                                        # 전체 프롬프트 텍스트의 시작에 특별 토큰을 넣음. (LLaMA-3 채팅 포맷에서 전체 대화의 시작을 알림)
        for msg in messages:                                                                # 각 메시지(시스템, 유저, 어시스턴트 등)에 대해 아래 작업을 반복
            role = msg["role"]                                                              # 메시지의 역할(예: "system", "user", "assistant")을 꺼냄.
            content = msg["content"].strip()                                                # 메시지 본문(내용)을 꺼내고, 앞뒤 공백을 제거.
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"    # 각 메시지를 LLaMA-3 채팅 포맷에 맞춰 특별 토큰으로 감싸 하나의 프롬프트 문자열로 이어붙임.
                                                                                            # <|start_header_id|>역할<|end_header_id|> 각 메시지의 역할(시스템/유저/어시스턴트) 표시
                                                                                            # \n내용<|eot_id|> 실제 메시지 내용과 그 끝을 나타내는 토큰

        text = prompt.strip()   # 완성된 전체 프롬프트 텍스트의 앞뒤 공백을 제거해서 최종적으로 저장


        # 프롬프트 전체 텍스트를 모델이 이해할 수 있는 숫자 토큰(input_ids)으로 변환.
        tokenized = tokenizer(
            text,
            truncation=True,             # 입력이 max_seq_length(최대 길이)를 넘으면 자동으로 잘라냄.
            max_length=max_seq_length,   # 입력 토큰의 최대 개수를 제한.
            padding=False,               # 이 단계에서는 패딩(길이 맞추기)을 하지 않습니다.(나중에 배치에서 패딩함)
            return_tensors=None,         # 결과를 일반 파이썬 리스트 형태로 반환.
        )

        input_ids = tokenized["input_ids"]             # 텍스트가 숫자 토큰 리스트로 바뀐 결과. 예: [128, 5551, 29871, ...] (각 숫자는 단어나 특수토큰에 해당)
        attention_mask = tokenized["attention_mask"]   # input_ids에서 실제 데이터(=1), 패딩(=0)을 구분하는 마스크.여기선 모두 1로만 채워짐(아직 패딩이 없으므로).
        labels = [-100] * len(input_ids)               # input_ids와 동일한 길이의 리스트를 -100으로 채움. -100은 PyTorch에서 "이 위치는 손실 계산(학습)에서 무시하라"는 의미임.
                                                       # 이후 assistant(답변) 구간에서만 실제 정답 토큰값으로 바뀜.


        # 답변(assistant) 부분이 시작되는 지점의 특수 토큰 문자열. 이 토큰 뒤부터 모델의 정답(레이블)로 사용할 구간이 시작
        # assistant_header 문자열을 숫자 토큰 시퀀스로 변환. 나중에 input_ids 안에서 이 시퀀스가 어디에 있는지 찾아 "정답 시작 위치"로 사용.
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)

        # 한 메시지(assistant 답변)가 끝났음을 표시하는 특수 토큰 문자열
        # eot_token도 숫자 토큰 시퀀스로 변환. 나중에 답변의 끝 위치를 정확히 찾는 데 사용.
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)


        # input_ids(토큰 시퀀스)에서 assistant(정답) 구간만 정확하게 찾아서 labels에 복사해 모델이 질문-답변 데이터에서“정답(답변 부분)만 학습”하도록 레이블을 세팅.
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):                 # input_ids 리스트에서 assistant_tokens(=assistant 시작 토큰 시퀀스)가 어디에 있는지 찾기
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens: # 현재 위치(i)부터 assistant 시작 토큰 시퀀스와 정확히 일치하는 부분을 찾으면 아래 실행
                start = i + len(assistant_tokens)                          # assistant 답변의 "실제 내용"이 시작되는 토큰 위치를 저장.
                end = start                                                # start부터 eot_tokens(=답변 끝 토큰 시퀀스)가 처음 나올 때까지 end를 증가시켜 답변의 끝 위치를 찾음.
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):                                # 답변(assistant)의 본문 구간을 labels에 복사해서 정답으로 사용. 이 구간만 손실 계산(모델 학습)에 실제로 반영.
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):                # eot_tokens(=답변 종료 특수토큰)도 정답에 포함. 모델이 어디서 답변을 끝내야 하는지도 학습
                    labels[j] = input_ids[j]
                break                                                      # 첫 번째 assistant 구간만 처리하고, 그 뒤는 무시(중복 적용 방지).
            i += 1                                                         # 다음 위치로 이동하며 assistant 시작 시퀀스를 계속 탐색.


        # 이 샘플의 input_ids, attention_mask, labels를 배치(new_batch)에 저장.
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 처리
    max_length = max(len(ids) for ids in new_batch["input_ids"])              # 미니배치 안에서 가장 긴 input_ids의 길이를 구함. 배치 내 모든 입력이 이 길이에 맞게 통일될 예정.
    for i in range(len(new_batch["input_ids"])):                              # 배치의 각 샘플에 대해 아래 과정을 반복
        pad_len = max_length - len(new_batch["input_ids"][i])                 # 현재 샘플의 길이가 max_length보다 짧으면, 부족한 만큼 pad_len을 계산..
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)  # input_ids의 끝에 pad_token_id(패딩 토큰)를 pad_len만큼 추가해서 길이를 맞춤.
        new_batch["attention_mask"][i].extend([0] * pad_len)                  # attention_mask의 끝에도 0을 pad_len만큼 추가.(패딩된 부분은 0, 실제 데이터는 1).
        new_batch["labels"][i].extend([-100] * pad_len)                       # labels의 끝에는 -100을 pad_len만큼 추가.(패딩된 부분은 학습에서 무시).

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])                             # 각 리스트를 PyTorch 텐서로 변환해서 모델에 바로 입력할 수 있게 만듦.

    return new_batch                                                          # 완성된 미니배치(모든 샘플의 길이가 같고, 텐서 형태로 변환된 딕셔너리)를 반환


* **new\_batch 딕셔너리 준비**

  * `input_ids`, `attention_mask`, `labels` 리스트를 만듭니다.

* **for example in batch:**

  * 미니배치 안에 있는 각 예제를 하나씩 처리합니다.

* **프롬프트 합치기 (LLaMA-3 채팅 템플릿)**

  * 각 메시지(system/user/assistant)를
    `<|start_header_id|>역할<|end_header_id|>\n내용<|eot_id|>`
    형식으로 이어붙입니다.
  * 전체 프롬프트는 `<|begin_of_text|>`로 시작합니다.

* **토큰화**

  * 프롬프트 전체를 tokenizer로 숫자 토큰(=input\_ids)과 attention\_mask로 변환합니다.
  * labels는 일단 모두 -100으로 채웁니다(학습 무시 구간).

* **assistant(정답) 구간 찾기**

  * `assistant_header` 토큰 시퀀스를 input\_ids에서 찾아 시작 위치를 구합니다.
  * 시작 이후 처음 나오는 `<|eot_id|>`까지가 답변 구간입니다.
  * 그 구간은 labels에 실제 input\_ids 값을 복사(즉, 정답으로 처리)
  * `<|eot_id|>` 토큰도 레이블에 포함시켜 모델이 종료 신호까지 예측하도록 합니다.

* **배치 패딩**

  * 한 배치 안에서 가장 긴 input\_ids 길이에 맞춰
    input\_ids: pad\_token\_id,
    attention\_mask: 0,
    labels: -100
    으로 뒷부분을 채웁니다.

* **텐서 변환**

  * PyTorch 텐서로 바꿔서 모델에 입력할 수 있는 형태로 만듭니다.

#### 요약

* 이 함수는 “배치 단위 데이터 → LLM 입력용 토큰/마스크/정답”을 한 번에 만들어줍니다.
* labels를 assistant(정답) 구간에만 활성화해,
  **모델이 질문과 답변이 섞인 긴 텍스트에서
  정답 부분만 똑똑하게 학습하도록** 도와줍니다.

**정리:**
data\_collator는 LLM 튜닝에서
“질문-답변 구조의 데이터를 모델이 잘 이해하고,
딱 필요한 부분만 정답으로 맞추게 해주는”
필수 전처리 함수입니다.


In [ ]:
example = train_dataset[0]          # 학습 데이터셋에서 첫 번째 예시(example)를 꺼냠.
batch = data_collator([example])    # 하나의 예시만 넣어서(리스트로) data_collator 함수를 호출. 결과는 모델이 입력으로 사용할 수 있는 "미니배치(딕셔너리 형태)"로 변환

print("batch:")
print("input_ids 크기:", batch["input_ids"].shape)              # input_ids(숫자 토큰 시퀀스)의 크기를 출력. (배치 크기 1, 시퀀스 길이 3388)
print("attention_mask 크기:", batch["attention_mask"].shape)    # attention_mask의 크기를 출력.input_ids와 동일한 shape
print("labels 크기:", batch["labels"].shape)                    # labels(정답 토큰 시퀀스)의 크기를 출력, 역시 input_ids와 동일한 shape

batch:
input_ids 크기: torch.Size([1, 3388])
attention_mask 크기: torch.Size([1, 3388])
labels 크기: torch.Size([1, 3388])


In [ ]:
# 배치 데이터(batch)에서 첫 번째 예시의 input_ids(숫자 토큰 시퀀스)를 리스트 형태로 출력
# 이 리스트는 LLM(언어모델) 입력용으로 변환된 **모든 토큰의 ID(숫자)**로 이루어져 있음.
# 각 숫자는 프롬프트 내 단어, 구두점, 특수토큰 등 하나하나에 해당
print('input_ids 인코딩 결과:')
print(batch["input_ids"][0].tolist())

input_ids 인코딩 결과:
[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 56773, 125441, 110714, 117294, 90519, 124338, 82818, 120378, 43139, 11, 41820, 110257, 109760, 19954, 82273, 113760, 127923, 102893, 11, 107536, 117294, 67236, 57575, 73653, 61139, 18918, 106589, 93292, 17835, 55000, 58126, 111964, 44005, 15592, 75086, 27796, 80052, 627, 54059, 54542, 67890, 108308, 18359, 64857, 30446, 30426, 67890, 115061, 92769, 1473, 12, 111964, 34804, 64857, 30446, 30426, 27437, 57575, 107364, 34804, 109842, 19954, 62398, 97237, 73653, 114839, 34983, 92769, 13, 27437, 19954, 108838, 109842, 34804, 58935, 103778, 16582, 109745, 67890, 32179, 96318, 22035, 96677, 51402, 627, 12, 111964, 57575, 59777, 27797, 44005, 115155, 13094, 91786, 33390, 11, 64857, 30446, 30426, 95713, 117294, 21028, 85721, 48424, 7, 103415, 25, 4416, 5349, 16, 21128, 4416, 5349, 17, 30716, 17835, 106589, 93292, 18918, 125538, 34983, 56773, 51402, 627, 12, 27437, 21028, 106248, 27796, 81673, 85721, 48424, 16969, 115489, 

### 토크나이즈된 입력을 다시 텍스트로 디코딩


* `tokenizer.decode(...)`

  * 숫자로 변환된 input\_ids(토큰 시퀀스)를 다시 원래의 자연어 텍스트로 변환합니다.
  * 즉, "숫자 토큰 리스트 → 텍스트"로 복원합니다.

* `batch["input_ids"][0].tolist()`

  * 방금 만든 배치에서 첫 번째 샘플의 input\_ids를 리스트 형태로 꺼냅니다.

* `skip_special_tokens=False`

  * 특별 토큰(예: `<|begin_of_text|>`, `<|start_header_id|>` 등)을 그대로 보이게 합니다.
  * 이 옵션이 True면 이런 토큰들이 빠집니다.

* `clean_up_tokenization_spaces=False`

  * 디코딩 후 공백 등 자동 정리를 하지 않습니다.


아래 코드는
**모델에 입력되는 숫자 토큰이
정확히 어떤 텍스트로 변환되는지 직접 눈으로 확인**할 수 있게 해줍니다.
(토큰화·디토큰화가 원하는 대로 동작하는지 실습할 때 매우 유용한 단계입니다.)


In [ ]:
text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 텍스트 디코딩:")
print(text)


input_ids 텍스트 디코딩:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>
질문: 멕시코 연방 정부는 메탈클래드에게 얼마를 보상하라고 하였나?

docs:
doc1: 소송가액만 약 5조원에 달하는 론스타펀드가 한국 정부를 상대로 제기한 투자자-국가소송(ISD)의 첫 심리가 15일(현지시간) 미국 워싱턴 국제투자분쟁해결센터(ICSID)에서 열린다. ISD는 1987년 애플 홍콩법인이 스리랑카 정부를 상대로 낸 것을 시작으로 주로 선진국 투자자들이 개발도상국 정부를 제소해왔다.한국 정부가 ISD에 휘말린 것은 이번이 처음이다. 결과가 미칠 파장이 만만치 않은 만큼 정부는 국무조정실 주도로 태스크포스를 구성했다. 세금 회피를 위해 벨기에 등에 페이퍼컴퍼니를 세운 뒤 외환은행을 인수한 론스타는 소송 주체도 페이퍼컴퍼니를 내세운 것으로 알려져 정부는 이 점을 집중 공략할 것으로 전해졌다.○한국 정부의 첫 ISD 사건론스타가 ICSID에 중재를 신청한 것은 2012년 11월21일이다. 신청인(원고)은 LSF-KEB홀딩스, 스타홀딩

In [ ]:
# 배치 데이터(batch)에서 첫 번째 예시의 labels(정답 토큰 시퀀스)를 리스트 형태로 출력
# 이 리스트는 input_ids와 같은 길이이며, 모델이 실제로 정답을 맞춰야 하는 토큰 위치에만 input_ids 값이 들어있고,
# 그 외 구간(질문/프롬프트 등)은 모두 -100으로 채워져 있음
print('레이블 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [ ]:
# <|eot_id|>라는 특별 토큰을 토크나이저로 숫자 토큰 ID로 변환.
# add_special_tokens=False 옵션을 통해 추가적인 시작/끝 토큰 없이 딱 해당 특수 토큰만 ID로 인코딩함.
# 결과는 input_ids 리스트(길이 1). [0]을 붙이면 실제 <|eot_id|>에 해당하는 숫자 토큰 값을 가져옴.
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]
eos_token  # 이 변수는 모델에서 "한 메시지(assistant 답변)가 끝났다"는 것을 표시하는 특수 토큰 ID를 담음.

128009

In [ ]:
# labels(정답 토큰 시퀀스)에서 -100이 아닌 값만 추려서 새로운 리스트를 만듦.
# -100은 "이 위치는 학습/평가에서 무시"하라는 의미이므로, 실제 답변 구간만 남게 됨.
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

# label_ids(정답 토큰 리스트)를 다시 사람이 읽을 수 있는 **텍스트(문자열)**로 변환.
decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,            # 특별 토큰도 그대로 보이게 함.
    clean_up_tokenization_spaces=False    # 디코딩 후 공백 등 자동 정리를 하지 않음.
)

print("\nlabels 디코딩 결과:")
print(decoded_labels)  # 레이블(정답) 시퀀스에서 학습·평가에 사용되는 답변 구간만 텍스트로 복원해 정확하게 원하는 정답만 추출되었는지 검증


labels 디코딩 결과:
멕시코 연방 정부는 메탈클래드에게 1,668만 5천 달러를 배상하도록 결정하였으며, 이후 캐나다 브리티시컬럼비아주 대법원에서 이 금액이 110만 달러 감액되어 최종적으로 1,560만 달러의 배상 판결이 확정되었습니다[[doc5]].<|eot_id|>


In [ ]:
# 데이터의 최대 길이 제한
max_seq_length = 8192

# 0번과 1번 데이터의 길이 확인
example0 = train_dataset[0]
example1 = train_dataset[1]

# 개별 길이 확인 (토큰화 후)
tokenized0 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example0["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
tokenized1 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example1["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)

print(f"0번 데이터 길이: {len(tokenized0['input_ids'])}")
print(f"1번 데이터 길이: {len(tokenized1['input_ids'])}")

# 배치로 처리하여 어텐션 마스크 비교
batch = data_collator([example0, example1])
print("\n배치 처리 후:")
print(f"입력 ID 형태: {batch['input_ids'].shape}")
print(f"어텐션 마스크 형태: {batch['attention_mask'].shape}")

# 각 샘플의 어텐션 마스크 합계 (실제 토큰 수 확인)
print(f"0번 샘플 어텐션 마스크 합계: {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계: {batch['attention_mask'][1].sum().item()}")

# 0번 샘플과 1번 샘플의 어텐션 마스크가 다른지 확인
masks_different = not torch.equal(batch['attention_mask'][0], batch['attention_mask'][1])
print(f"\n0번과 1번 샘플의 어텐션 마스크가 다른가요? {masks_different}")

# 어텐션 마스크 패턴 시각화 (처음 20개와 마지막 20개 토큰)
print("\n0번 샘플 어텐션 마스크:", batch['attention_mask'][0].tolist())
print("1번 샘플 어텐션 마스크:", batch['attention_mask'][1].tolist())

# 배치 내에서 가장 긴 시퀀스 길이 구하기
max_length_in_batch = max(len(tokenized0['input_ids']), len(tokenized1['input_ids']))
print(f"\n배치 내 최대 길이: {max_length_in_batch}")
print(f"0번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][1].sum().item()}")
print(f"0번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][0].sum().item()}")
print(f"0번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][0] == 0).sum().item()}")
print(f"1번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][1].sum().item()}")
print(f"1번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][1] == 0).sum().item()}")
# 결과 검증: 긴 샘플은 모든 어텐션 마스크가 1이고, 짧은 샘플은 일부만 1이어야 함

0번 데이터 길이: 3388
1번 데이터 길이: 3823

배치 처리 후:
입력 ID 형태: torch.Size([2, 3823])
어텐션 마스크 형태: torch.Size([2, 3823])
0번 샘플 어텐션 마스크 합계: 3388
1번 샘플 어텐션 마스크 합계: 3823

0번과 1번 샘플의 어텐션 마스크가 다른가요? True

0번 샘플 어텐션 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# TRL(Transformers Reinforcement Learning) 라이브러리에서 SFT(지도 미세조정)를 위한 Trainer 클래스를 불러옴.
# 이 Trainer는 LLM 파인튜닝을 매우 간단하게 관리할 수 있도록 설계된 도구
from trl import SFTTrainer

# Hugging Face Hub에 모델을 업로드할 때 필요한 로그인 함수
from huggingface_hub import login

# Hugging Face 계정의 접근 토큰을 사용해 인증을 진행.
# 이 과정을 거쳐야 push_to_hub=True로 설정한 경우 모델 업로드가 가능
login(token='')

# SFTTrainer 객체
trainer = SFTTrainer(
    model=model,                      # 파인튜닝할 사전학습 LLM
    args=args,                        # SFTConfig로 정리한 학습 세팅(하이퍼파라미터, 저장, 허브 옵션 등)
    train_dataset=train_dataset,      # 학습 데이터셋
    data_collator=data_collator,      # 위에서 만든 data_collator 함수(토큰/마스크/레이블 생성)
    peft_config=peft_config           # LoRA 어댑터 설정(파라미터 효율적 파인튜닝 적용)
)

In [ ]:
# 앞서 준비한 SFTTrainer(지도 미세조정 Trainer) 객체에서 실제 파인튜닝 학습을 시작하는 함수.
# args에 설정한 하이퍼파라미터, 데이터셋, data_collator, LoRA 설정 등이 모두 반영되어 전체 학습 과정이 자동으로 진행.
# 전체 데이터셋을 주어진 에포크 수만큼 반복 학습
# 일정 step마다 로깅 및 체크포인트 저장
# validation, early stopping 등 추가 옵션도 내부적으로 지원
# 학습이 끝나면 output_dir(또는 hub)에 최종 모델 저장

trainer.train()

Step,Training Loss
10,0.770200
20,0.602100
30,0.536700
40,0.472200
50,0.523300
60,0.548000
70,0.479400
80,0.487300
90,0.537200
100,0.475700


TrainOutput(global_step=846, training_loss=0.41533792004799447, metrics={'train_runtime': 5019.7791, 'train_samples_per_second': 0.673, 'train_steps_per_second': 0.169, 'total_flos': 6.007696178425037e+17, 'train_loss': 0.41533792004799447})

In [ ]:
# 현재까지 학습된 모델(파라미터, 토크나이저, LoRA 어댑터 등)을 output_dir에 저장.
# 이 명령을 실행하면 학습을 중간에 멈추더라도 나중에 이어서 학습하거나, 바로 추론·평가에 활용할 수 있음.
# 체크포인트 저장과는 별도로 마지막 모델 상태를 명확히 파일로 남길 때 사용
# 저장 위치는 SFTConfig에서 지정한 output_dir
# 이 함수 호출 후 **trainer.push_to_hub()**로 Hub 업로드를 연계할 수 있음
# 학습을 마친 LLM(및 어댑터 등)을 디스크 또는 클라우드 저장소에 최종적으로 안전하게 저장

trainer.save_model()

In [ ]:
# 방금 저장한 모델(및 필요한 부속 파일들)을 **Hugging Face Hub(클라우드 저장소)**로 업로드하는 명령.
# commit_message는 업로드할 때 남기는 설명(버전 관리 기록용)
# 이 명령을 실행하면 누구나(또는 권한이 있는 사용자만) 인터넷에서 모델을 쉽게 불러오거나, 공유, 재사용할 수 있음.
# 업로드 대상은 SFTConfig에서 지정한 hub_model_id.

# trainer.push_to_hub(commit_message="Upload LoRA adapter")

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nugunaai/llama3-8b-rag-ko/commit/04dcba27953beca1ef5168b872bf9556008aa791', commit_message='Upload LoRA adapter', commit_description='', oid='04dcba27953beca1ef5168b872bf9556008aa791', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nugunaai/llama3-8b-rag-ko', endpoint='https://huggingface.co', repo_type='model', repo_id='nugunaai/llama3-8b-rag-ko'), pr_revision=None, pr_num=None)

In [ ]:
# 평가(테스트)용 프롬프트(입력)와 정답(라벨) 리스트를 각각 만듦.
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:                                                            # 테스트 데이터셋의 각 샘플에서 messages(대화 내용 리스트)를 하나씩 꺼냄.
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)      # messages 리스트를 LLaMA-3 채팅 포맷 텍스트로 변환
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + \
        '<|start_header_id|>assistant<|end_header_id|>\n'                                            # assistant(정답) 답변이 시작되기 전까지의 텍스트를 프롬프트로 만듦.
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]  # assistant 구간의 시작 이후부터 <|eot_id|>(답변 끝) 이전까지의 텍스트만 추출해 정답으로 만듦.
    prompt_lst.append(input)                                                                         # 입력(프롬프트) 텍스트를 리스트에 저장
    label_lst.append(label)                                                                          # 정답(답변) 텍스트를 리스트에 저장

In [ ]:
print(prompt_lst[200])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

질문: 현대미술에서 일본의 모노하 운동과 한국의 추상 표현주의가 어떤 방식으로 서로 영향을 주고 있으며, 이 두 장르의 작가들이 사용하는 재료와 주제는 어떻게 다르게 나타나는지 비교해 주실 수 있습니까?

docs:
doc1: 제2차 세계대전에 패한 직후 일본 화단에는 형태를 극도로 단순하게 표현하는 서구의 ‘미니멀 아트’를 동양적으로 재해석한 전위미술 장르 ‘모노하(物派)’가 등장했다. 미니멀 아트가 감정을 철저히 배제한 채 작업하는 ‘결과의 예술’이라면, 모노하는 사물을 있는 그대로 놓아두는 것을 통해 사물과 공간, 위치, 상황, 관계 등에 접근하는 예술이다.구체적으로는 돌, 철, 나무 등 재료의 성격을 그대로 드러낸다든가 그림의 원재료인 점, 선, 면에 주목하는 식이다. 일본에서 활동한 고(故) 곽인식 화백(1919~1988)은 이 같은 모노하 운동을 주도하며 1960~1970년대 한국과 일본의 현대미술에 큰 영향을 미쳤다.일본 모노하의 선구적 작가인 곽 화백의 예술세계를

In [ ]:
print(label_lst[200])


현대미술에서 일본의 모노하 운동과 한국의 추상 표현주의는 서로 다른 접근 방식과 주제를 가지고 있지만, 두 장르 모두 물질과 형태에 대한 탐구를 중심으로 하고 있습니다.

모노하 운동은 형태를 극도로 단순하게 표현하며, 사물을 있는 그대로 놓아두는 방식으로 사물과 공간, 위치, 상황, 관계 등을 탐구합니다. 이 운동은 돌, 철, 나무와 같은 재료의 성격을 그대로 드러내는 작업을 특징으로 하며, 일본의 고(故) 곽인식 화백이 이 운동을 주도했습니다[[doc1]].

반면, 한국의 추상 표현주의는 감정과 색, 선율을 통해 내면의 감정을 표현하는 데 중점을 둡니다. 최홍순 화백은 그의 작품에서 색과 선율을 통해 분노, 불안, 공포 등을 표현하며, 민화적 이미지와 음악적 요소를 결합하여 독특한 주제를 다룹니다[[doc2]].

재료와 주제의 차이점으로는, 모노하 운동이 물질의 본질을 탐구하는 데 중점을 두는 반면, 한국의 추상 표현주의는 감정의 표현과 음악적 요소를 강조합니다. 모노하의 작가들은 주로 자연 재료의 물성을 탐구하는 반면, 한국의 추상 표현주의 작가들은 색과 형태를 통해 감정과 이야기를 전달하는 데 집중합니다[[doc1]], [[doc2]]. 

이러한 차이점은 두 장르가 서로 다른 문화적 배경과 미적 가치관을 반영하고 있음을 보여줍니다.


In [ ]:
# PEFT 라이브러리에서 파인튜닝(LORA 등 PEFT)이 적용된 생성형 언어모델을 자동으로 불러오는 클래스를 가져옴.
from peft import AutoPeftModelForCausalLM

# Hugging Face에서 토크나이저와 추론 파이프라인 클래스를 가져옴.
from transformers import AutoTokenizer, pipeline

# LoRA 등으로 파인튜닝된 모델(또는 체크포인트)의 저장 경로. 실습 예시에서는 특정 체크포인트를 불러와 추론에 사용.
peft_model_id = "llama3-8b-rag-ko/checkpoint-846"

# LoRA 등 PEFT가 적용된 사전학습 LLM을 지정한 경로에서 불러옴.
# device_map="auto": GPU/CPU 환경에 따라 자동으로 할당
# torch_dtype=torch.float16:
# bfloat16(16비트 부동소수점) 형식으로 메모리/속도를 최적화
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.bfloat16)

# fine_tuned_model과 같은 토크나이저를 연결해서 "텍스트 생성(챗봇, 요약 등)" 추론 파이프라인을 만듭니다.
# 이 파이프라인 객체로 프롬프트 입력→모델 응답 생성까지 한 번에 처리
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]
eos_token

128009

In [ ]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
for prompt, label in zip(prompt_lst[200:202], label_lst[200:202]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
현대미술에서 일본의 모노하 운동과 한국의 추상 표현주의는 서로 다른 방식으로 발전하고 있으며, 이 두 장르의 작가들이 사용하는 재료와 주제는 다음과 같이 비교할 수 있습니다.

모노하 운동은 일본에서 1960~1970년대에 등장한 전위미술 장르로, 사물을 그대로 놓아두는 방식으로 사물과 공간, 위치, 상황, 관계에 접근합니다. 이 운동의 선구적 작가인 곽인식 화백은 나무, 쇠, 유리 등의 물성을 화면과 형상에 반영하는 작품을 선보였습니다. 그의 작품은 단색과 선, 면에 주목하며, 특유의 공간과 빛의 세계를 연출하는 것이 특징입니다[[doc1]].

반면, 한국의 추상 표현주의는 최홍순 화백을 중심으로 발전하였습니다. 최 화백은 캔버스에 빛과 생명, 우주를 색의 물결로 묘사하는 화가로 알려져 있으며, '생명의 율동'을 주제로 한 작품을 선보입니다. 그의 작품에서는 다양한 색깔의 영혼이 있는 사물을 소재로 삼고, 음악적인 소재를 통해 자유분방한 표현을 추구합니다[[doc2]].

재료의 사용에 있어서, 모노하 운동은 주로 돌, 철, 나무와 같은 자연 재료를 사용하는 반면, 추상 표현주의는 캔버스와 같은 기존의 미술 매체를 활용합니다. 또한, 모노하 운동은 단색과 선, 면에 중점을 두는 반면, 추상 표현주의는 다양한 색채와 선을 통해 생동감 있는 표현을 시도합니다.

주제의 경우, 모노하 운동은 사물의 본질을 드러내는 데 중점을 두는 반면, 추상 표현주의는 생명력과 율동을 주제로 삼아 음악적 요소를 강조합니다. 이러한 차이는 두 장르가 서로 다른 시각과 접근 방식을 보여주는 것입니다.
    label:

현대미술에서 일본의 모노하 운동과 한국의 추상 표현주의는 서로 다른 접근 방식과 주제를 가지고 있지만, 두 장르 모두 물질과 형태에 대한 탐구를 중심으로 하고 있습니다.

모노하 운동은 형태를 극도로 단순하게 표현하며, 사물을 있는 그대로 놓아두는 방식으로 사물과 공간, 위치, 상황, 관계 등을 탐구합니다. 이 운동은 돌, 철, 나무와 같

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
교황 첼레스티노 1세와 교황 식스토 3세의 재위 기간 동안 교회에서 다룬 주요 이슈와 그에 따른 결정은 다음과 같습니다.

첼레스티노 1세는 네스토리우스의 교리와 관련된 논쟁이 주요 이슈였습니다. 그는 네스토리우스가 펠라기우스주의에 빠진 율리아누스와 가깝다는 사실을 알게 되자, 알렉산드리아의 치릴로 주교에게 네스토리우스에 대한 조사를 지시하였습니다. 이 조사를 바탕으로 430년 로마 시노드에서 첼레스티노 1세는 네스토리우스를 단죄하였고, 네스토리우스는 자신의 오류를 인정하여 철회할 것을 지시받았습니다. 이후, 에페소 공의회에서 네스토리우스의 해임이 확인되었습니다[[doc1]].

식스토 3세는 성모 마리아의 신앙적 의미와 관련된 논쟁이 주요 이슈였습니다. 그는 431년 에페소 공의회에서 성모 마리아를 '하느님의 어머니'로 공식적으로 선포하였고, 이는 그리스도의 신성과 인성의 결합에 대한 논쟁을 해결하는 데 기여하였습니다. 또한, 식스토 3세는 알렉산드리아의 치릴로 주교와 시리아 교회 사이의 평화 관계 회복을 위해 노력하였습니다[[doc3]].

이러한 결정들은 교회의 신앙적 정체성을 강화하는 데 기여하였으며, 교황의 권위와 교회의 사목권 확장에도 영향을 미쳤습니다. 첼레스티노 1세의 네스토리우스에 대한 결정은 교회의 교리적 일관성을 유지하는 데 중요한 역할을 했고, 식스토 3세의 마리아 신앙에 대한 결정은 교회의 신앙적 가르침을 정립하는 데 기여하였습니다.

또한, 교황 첼레스티노 1세의 재위 기간 동안 교황 식스토 3세의 재위 기간 동안 교황의 사목권이 확장되는 결정은 교회의 지역적 영향력을 강화하는 데 기여하였습니다. 식스토 3세는 교황의 사목권을 일리리아까지 뻗쳐, 교회의 지역적 영향력을 더욱 공고히 하였습니다[[doc3]].

이러한 결정들은 교회의 신앙적, 사목적 영향뿐만 아니라, 교황의 권위와 교회의 지역적 영향력에도 긍정적인 영향을 미쳤습니다.
    label:

교황 첼레스티노 1세와 교황 식스토 3세의 재위 기간 동안